In [1]:
################################### setting path ###################################
import sys
sys.path.append('../')
sys.path.append('../../')
################################### import libs ###################################
from  pytorchyolo import  models_split_large
# from pytorchyolo import models
import numpy as np
import torch
from torchinfo import summary


In [56]:
cfg_path = "../ckpt/yolov3.cfg"
model_path = "../ckpt/stmarc_full.pth"

split_layer=12


In [57]:
model = models_split_large.load_model(cfg_path, model_path)
model.set_split_layer(split_layer) # layer <7
model = model.eval()
# summary(model, input_size=(1, 3, 416, 416))

KeyboardInterrupt: 

In [54]:
input_tensor = torch.rand(1,3, 416,416).cuda()
head_output = model(input_tensor,1)
head_output.shape

-3
4
-3
8
-3
11


torch.Size([1, 128, 104, 104])

In [55]:
output = model(head_output,2)

-3
3


IndexError: list index out of range

In [8]:
output

tensor([[[ 15.6466,  17.2949,  54.5350,  ...,   0.4822,   0.4822,   0.4822],
         [ 47.6612,  17.3243,  54.4646,  ...,   0.4824,   0.4824,   0.4824],
         [ 79.6610,  17.3240,  54.4667,  ...,   0.4824,   0.4824,   0.4824],
         ...,
         [395.9433, 412.0581,  18.4249,  ...,   0.4920,   0.4920,   0.4920],
         [403.9251, 412.0674,  18.6911,  ...,   0.4913,   0.4913,   0.4913],
         [411.9394, 412.0968,  18.3902,  ...,   0.4870,   0.4870,   0.4870]]],
       device='cuda:0', grad_fn=<CatBackward0>)

In [23]:
# summary(model, input_size=(1, 3, 416, 416), dtypes=[torch.float16])

In [28]:
model_int8 = torch.quantization.quantize_dynamic(
    model,  # your model
    {torch.nn.Linear},  # layers to quantize
    dtype=torch.qint8
)

In [32]:
x = torch.randn(1, 3, 416, 416)
x_q = torch.quantize_per_tensor(x, scale=0.1, zero_point=128, dtype=torch.quint8)

summary(model_int8, input_size=(1, 3, 416, 416))

Layer (type:depth-idx)                   Output Shape              Param #
Darknet                                  [1, 256, 26, 26]          --
├─ModuleList: 1-1                        --                        8,459,262
│    └─Sequential: 2-1                   [1, 16, 416, 416]         --
│    │    └─Conv2d: 3-1                  [1, 16, 416, 416]         432
│    │    └─BatchNorm2d: 3-2             [1, 16, 416, 416]         32
│    │    └─LeakyReLU: 3-3               [1, 16, 416, 416]         --
│    └─Sequential: 2-2                   [1, 16, 208, 208]         --
│    │    └─MaxPool2d: 3-4               [1, 16, 208, 208]         --
│    └─Sequential: 2-3                   [1, 32, 208, 208]         --
│    │    └─Conv2d: 3-5                  [1, 32, 208, 208]         4,608
│    │    └─BatchNorm2d: 3-6             [1, 32, 208, 208]         64
│    │    └─LeakyReLU: 3-7               [1, 32, 208, 208]         --
│    └─Sequential: 2-4                   [1, 32, 104, 104]         --
│   